In [66]:
'''
Model oluşturma ve oluşturulan modelin test edilmesi

reduced_data.csv dosyasını okur ve bu dosya ile 4 tane model oluşturur.
Test dosyalarını okur ve özniteliklerin sıklıklarıyla tablo oluşturur.

Kullanılan sınıflandırma algoritmalar:
1- Linear Support Vector 
2- K-Nearest Neighbors
3- Gaussian Naive Bayes
4- Stochastic Gradient Descent
5- Rocchio
6- Multinomial Naive Bayes

Bu algoritmalardan en iyi sonucu veren Multinominal Naive Bayes oldu.

'''

'\nModel oluşturma ve oluşturulan modelin test edilmesi\n\nreduced_data.csv dosyasını okur ve bu dosya ile 4 tane model oluşturur.\nTest dosyalarını okur ve özniteliklerin sıklıklarıyla tablo oluşturur.\n\nKullanılan sınıflandırma algoritmalar:\n1- Linear Support Vector \n2- K-Nearest Neighbors\n3- Gaussian Naive Bayes\n4- Stochastic Gradient Descent\n5- Rocchio\n6- Multinomial Naive Bayes\n\nBu algoritmalardan en iyi sonucu veren Multinominal Naive Bayes oldu.\n\n'

In [67]:
import pandas as pd
import os
import numpy as np
import grpc
import sys
import zemberek_grpc.language_id_pb2 as z_langid
import zemberek_grpc.language_id_pb2_grpc as z_langid_g
import zemberek_grpc.normalization_pb2 as z_normalization
import zemberek_grpc.normalization_pb2_grpc as z_normalization_g
import zemberek_grpc.preprocess_pb2 as z_preprocess
import zemberek_grpc.preprocess_pb2_grpc as z_preprocess_g
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
from sklearn.metrics import classification_report
import pickle

channel = grpc.insecure_channel('localhost:1234')
normalization_stub = z_normalization_g.NormalizationServiceStub(channel)
preprocess_stub = z_preprocess_g.PreprocessingServiceStub(channel)
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

def reportDf(report):
    report = [x.split(' ') for x in report.split('\n')]
    header = ['Class Name']+[x for x in report[0] if x!='']
    values = []
    for row in report[1:-5]:
        row = [value for value in row if value!='']
        if row!=[]:
            values.append(row)
    df = pd.DataFrame(data = values, columns = header)
    del df['support']
    avarages = ["ortalama"]
    for i in range(0,3):
        avarages.append(round(pd.to_numeric(df[['precision','recall','f1-score']].iloc[i]).mean(),2))  
    df.loc[df.index.max()+1] = avarages
    df = df.transpose()
    df.columns=list(df.iloc[0])
    df = df[1:]
    return df
def serializeModel(filename,model):
    pickle.dump(model, open(filename, 'wb'))

In [68]:
def tokenize(i):
    response = preprocess_stub.Tokenize(z_preprocess.TokenizationRequest(input=i))
    return response.tokens

def normalize(i):
    response = normalization_stub.Normalize(z_normalization.NormalizationRequest(input=i))
    return response

def analyze(i):
    response = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=i))
    return response;

def fix_decode(text):
    """Pass decode."""
    if sys.version_info < (3, 0):
        return text.decode('utf-8')
    else:
        return text

stop_words = list(map(lambda x: x.replace("\n","").encode("utf-8"), open("stop-words.txt",encoding="iso-8859-9").readlines()))
def isStopWord(word):
    if word == "UNK":
        return True
    return word in stop_words

def preprocess(document):
    tokenized = fix_decode(tokenize(normalize(document).normalized_input))
    output = []
    for i in tokenized:
        if i.type == 'Word':
            lemma = analyze(i.token).results[0].best.lemmas[0]
            #print("lemma(%s)=%s"%(i.token,lemma))
            if not isStopWord(lemma):
                output.append(str(lemma))
    return output


In [106]:
data = pd.read_csv("../reduced_data.csv")
train_filenames = data['files']
data = data[data.columns[~data.columns.isin(['Unnamed: 0','files'])]]
feature_names = data.columns

In [70]:
def getFiles(root_path):
    ignore = ['.DS_Store']
    folders = set(os.listdir(root_path))-set(ignore)
    txt_files = []
    for folder in folders:
        folder_path = "%s/%s/" % (root_path,folder)
        for file in set(os.listdir(folder_path))-set(ignore):
            file_path = "%s/%s/%s" %(root_path,folder,file)
            txt_files.append(file_path)
    return txt_files

def getPreprocessedDocument(path):
    f_in = open(path,encoding="iso-8859-9").read()
    doc = preprocess(f_in)
    data = [0]*(len(feature_names))
    for index,word in enumerate(feature_names):
        data[index] = doc.count(word)
    data[-1] = doc_type = path.split("/")[3]
    return data


test_files = getFiles(root_path = "../data/test")

In [71]:
test_data = []
test_file_names = []
for test_file in test_files:
    test_data.append(getPreprocessedDocument(test_file))
    test_file_names.append(test_file.split("/")[-1])

In [96]:
test = pd.DataFrame(test_data,columns=list(feature_names))
test.insert(0,'files',test_file_names)
test.to_csv("../test_data.csv")

In [97]:
x = data[data.columns[~data.columns.isin(['class','files'])]]
y = data['class']
test_x = test[test.columns[~test.columns.isin(['class','files'])]]
test_y = test['class']
target_names = list(set(test_y))

In [98]:
#LinearSVC
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support

svc_model = LinearSVC(random_state=12)
svc_model = svc_model.fit(x, y)
pred = svc_model.predict(test_x)
serializeModel("linear_svc.mdl",svc_model)
df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/linearsvc.csv")
df

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,spor,saglik,magazin,ekonomi,ortalama
precision,0.99,0.95,0.95,0.99,0.97
recall,0.96,0.97,0.97,0.96,0.96
f1-score,0.97,0.96,0.96,0.97,0.96


In [99]:
#Knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

neigh = KNeighborsClassifier(n_neighbors=5)

neigh.fit(x, y)
serializeModel("kneighbors.mdl",neigh)
pred = neigh.predict(test_x)

df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/kneighborsclassifier.csv")
df

,spor,saglik,magazin,ekonomi,ortalama
precision,0.96,0.47,1.00,0.80,0.77
recall,0.61,0.95,0.33,0.81,0.68
f1-score,0.75,0.63,0.49,0.81,0.61


In [100]:
#GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support

gnb = GaussianNB()
gnb = gnb.fit(x, y)
serializeModel("gaussian_nb.mdl",gnb)
pred = gnb.predict(test_x)

df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/gaussiannb.csv")
df

,spor,saglik,magazin,ekonomi,ortalama
precision,0.86,1.00,0.83,0.89,0.87
recall,0.88,0.82,0.88,0.99,0.91
f1-score,0.87,0.90,0.85,0.93,0.85


In [101]:
#SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support

sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd = sgd.fit(x, y)
serializeModel("sgdclassifier.mdl",sgd)
pred = sgd.predict(test_x)

df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/sgdclassifier.csv")
df

,spor,saglik,magazin,ekonomi,ortalama
precision,0.95,0.96,0.93,1.00,0.94
recall,0.94,0.95,0.97,0.97,0.96
f1-score,0.94,0.96,0.95,0.99,0.95


In [102]:
#Rocchio
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.metrics import precision_recall_fscore_support
rocchio = NearestCentroid(metric='euclidean')
rocchio = rocchio.fit(x, y)
serializeModel("rocchio.mdl",rocchio)
pred = rocchio.predict(test_x)
df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/rocchio.csv")
df

,spor,saglik,magazin,ekonomi,ortalama
precision,0.95,0.51,0.96,0.98,0.85
recall,0.76,0.96,0.65,0.62,0.71
f1-score,0.85,0.67,0.78,0.76,0.8


In [103]:
#MultinomialNB
from sklearn.naive_bayes import MultinomialNB
multinomialNB = MultinomialNB()
multinomialNB.fit(x,y)
serializeModel("multinominalnb.mdl",multinomialNB)
pred = multinomialNB.predict(test_x)
df = reportDf(classification_report(test_y, pred, target_names=target_names))
df.to_csv("../reports/multinomialnb.csv")
df

,spor,saglik,magazin,ekonomi,ortalama
precision,0.96,1.00,0.98,1.00,0.97
recall,0.97,0.99,0.99,0.99,0.99
f1-score,0.97,0.99,0.98,0.99,0.98


In [104]:
def getData(str_data):
    doc =  preprocess(str_data)
    print(doc)
    data = [0]*(len(feature_names)-1)
    for index,word in enumerate(list(set(feature_names)-set(['class']))):
        data[index] = doc.count(word)
    return data    

In [107]:
data.insert(0,"files",train_filenames)

In [108]:
merged = pd.concat([data,test])
merged.rename(index=str, columns={"class": "sınıf"})

,files,yakupoğlu,mayıs,mrı,çanta,fon,tekrar,hayat,gizle,diyarbakırspor,...,klavye,yukarı,fındık,cup,bulaş,damga,yatkın,ekip,azami,sınıf
0,145.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
1,16.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
2,144.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
3,106.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,ekonomi
4,149.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,ekonomi
5,65.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
6,5.txt,0,0,0,0,0,0,2,0,0,...,2,0,0,0,0,0,0,0,0,ekonomi
7,40.txt,0,0,0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
8,50.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi
9,54.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ekonomi


In [ ]:
merged.to_csv("../reports/terms.csv")

In [84]:
custom = getData("hahaha")
sgd.predict([custom])

['hah', 'ha']


array(['magazin'], dtype='<U7')

In [85]:
custom = getData("TCMB repo ihalesiyle piyasaya yaklaşık 16 milyar lira verdi.Türkiye Cumhuriyet Merkez Bankası (TCMB), bugün miktar yöntemiyle düzenlediği 29 Mayıs vadeli repo ihalesi ile piyasaya 15 milyar 999 milyon 999 bin 984 lira verdi.İhaleye 47 milyar 850 milyon liralık teklif geldi. İhalede en düşük, ortalama ve en yüksek basit faiz yüzde 24,00 olurken, en düşük, ortalama ve en yüksek bileşik faiz ise yüzde 27,05 seviyesinde gerçekleşti. ")
sgd.predict([custom])

['tcmb', 'repo', 'ihale', 'piyasa', 'yaklaşık', 'milyar', 'lira', 'ver', 'türkiye', 'cumhuriyet', 'merkez', 'banka', 'tcmb', 'bugün', 'miktar', 'yöntem', 'düzenle', 'mayıs', 'vade', 'repo', 'ihale', 'ile', 'piyasa', 'milyar', 'milyon', 'bin', 'lira', 'ver', 'ihale', 'milyar', 'milyon', 'lira', 'teklif', 'gel', 'ihale', 'en', 'düşük', 'ortalama', 've', 'en', 'yüksek', 'basit', 'faiz', 'yüz', 'ol', 'en', 'düşük', 'ortalama', 've', 'en', 'yüksek', 'bileşik', 'faiz', 'ise', 'yüz', 'seviye', 'gerçek']


array(['magazin'], dtype='<U7')